In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go

In [2]:
df = yf.download(tickers='BTC-USD', period='max', interval='1d', auto_adjust=False)
df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
df.reset_index(inplace=True)
df

[*********************100%***********************]  1 of 1 completed


,Date,Adj Close,Close,High,Low,Open,Volume
0,2014-09-17,457.334015,457.334015,468.174011,452.421997,465.864014,21056800
1,2014-09-18,424.440002,424.440002,456.859985,413.104004,456.859985,34483200
2,2014-09-19,394.795990,394.795990,427.834991,384.532013,424.102997,37919700
3,2014-09-20,408.903992,408.903992,423.295990,389.882996,394.673004,36863600
4,2014-09-21,398.821014,398.821014,412.425995,393.181000,408.084991,26580100
...,...,...,...,...,...,...,...
4145,2026-01-22,89462.453125,89462.453125,90258.960938,88438.445312,89378.523438,35549685694
4146,2026-01-23,89503.875000,89503.875000,91100.250000,88486.359375,89462.046875,38997586037
4147,2026-01-24,89110.734375,89110.734375,89811.609375,89044.289062,89506.148438,14558687712
4148,2026-01-25,86572.218750,86572.218750,89193.148438,86003.710938,89104.765625,36124986722


In [3]:
def rejection_signal(row):
    open_ = row.Open
    close = row.Close
    high = row.High
    low = row.Low

    body = abs(open_ - close)
    if body == 0:
        return 0

    # Bullish rejection
    if (
        open_ < close and
        (high - close) < body / 10 and
        (open_ - low) > body * 5
    ):
        return 2

    # Bearish rejection
    if (
        open_ > close and
        (high - open_) > body * 5 and
        (close - low) < body / 10
    ):
        return 1

    return 0
    
df["rejection_signal"] = df.apply(rejection_signal, axis=1)

In [4]:
def engulfing_signal(prev, curr):
    # Bullish engulfing
    if (
        curr.Close > prev.Open and
        curr.Open < prev.Close and
        prev.Open > prev.Close
    ):
        return 2

    # Bearish engulfing
    if (
        curr.Open > prev.Close and
        curr.Close < prev.Open and
        prev.Close > prev.Open
    ):
        return 1

    return 0

df["engulfing_signal"] = 0

for i in range(1, len(df)):
    df.loc[i, "engulfing_signal"] = engulfing_signal(df.iloc[i - 1], df.iloc[i])

In [5]:
mask = df.engulfing_signal == 1
next_candle = df.shift(-1)

up_count = ((next_candle.Close > next_candle.Open) & mask).sum()
down_count = ((next_candle.Close < next_candle.Open) & mask).sum()
total_count = mask.sum()

print(
    f"Up: {up_count / total_count * 100:.2f}%, ",
    f"Down: {down_count / total_count * 100:.2f}%, ",
    f"Total: {total_count}"
)

Up: 56.43%,  Down: 43.57%,  Total: 241


In [6]:
st = 0
dfpl = df.iloc[st:st + 150]

fig = go.Figure(data=[
    go.Candlestick(
        x=dfpl.index,
        open=dfpl.Open,
        high=dfpl.High,
        low=dfpl.Low,
        close=dfpl.Close,
        name= 'OHLC'
    )
])

fig.show()

In [7]:
def price_target(df, N=4):
    targets = np.zeros(len(df))

    for i in range(len(df) - N):
        future_avg = df.Close.iloc[i + 1:i + N + 1].mean()

        if future_avg > df.Close.iloc[i]:
            targets[i] = 2
        elif future_avg < df.Close.iloc[i]:
            targets[i] = 1

    return targets

N = 4
df["price_target"] = price_target(df, N)

In [ ]:
valid = df.engulfing_signal != 0

equal_count = (df.engulfing_signal == df.price_target)[valid].sum()
total_count = valid.sum()
match = equal_count / total_count

print(
    f"Match: {match * 100:.2f}%, ",
    f"Mismatch: {(1 - match) * 100:.2f}%"
)

Match: 46.72%,  Mismatch: 53.28%


In [9]:
df

,Date,Adj Close,Close,High,Low,Open,Volume,rejection_signal,engulfing_signal,price_target
0,2014-09-17,457.334015,457.334015,468.174011,452.421997,465.864014,21056800,0,0,1.0
1,2014-09-18,424.440002,424.440002,456.859985,413.104004,456.859985,34483200,0,0,1.0
2,2014-09-19,394.795990,394.795990,427.834991,384.532013,424.102997,37919700,0,0,2.0
3,2014-09-20,408.903992,408.903992,423.295990,389.882996,394.673004,36863600,0,0,2.0
4,2014-09-21,398.821014,398.821014,412.425995,393.181000,408.084991,26580100,0,0,2.0
...,...,...,...,...,...,...,...,...,...,...
4145,2026-01-22,89462.453125,89462.453125,90258.960938,88438.445312,89378.523438,35549685694,0,0,1.0
4146,2026-01-23,89503.875000,89503.875000,91100.250000,88486.359375,89462.046875,38997586037,0,0,0.0
4147,2026-01-24,89110.734375,89110.734375,89811.609375,89044.289062,89506.148438,14558687712,0,1,0.0
4148,2026-01-25,86572.218750,86572.218750,89193.148438,86003.710938,89104.765625,36124986722,0,0,0.0
